In [1]:
import os
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

2024-12-03 22:27:18.156175: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 22:27:18.159195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 22:27:18.169647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733239638.187539   18938 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733239638.192464   18938 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 22:27:18.209677: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [10]:
class KickBot:
  def __init__(self):
    work_dir = os.path.dirname(os.path.abspath('.'))
    
    self.lemmatizer = WordNetLemmatizer()
    self.intents = json.loads(open(os.path.join(work_dir, 'data/intents.json')).read())
    self.words = pickle.load(open(os.path.join(work_dir, 'model/words.pkl'), 'rb'))
    self.classes = pickle.load(open(os.path.join(work_dir, 'model/classes.pkl'), 'rb'))
    self.model = load_model(os.path.join(work_dir, 'model/chatbot.h5'))
    
    
  def clean_up_sentence(self, sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [self.lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    return sentence_words

  def bow(self, sentence, words, show_details=True):
    sentence_words = self.clean_up_sentence(sentence)
    bag = [0] * len(words)

    for s in sentence_words:
      for i, w in enumerate(words):
        if w == s:
          bag[i] = 1
          if show_details:
              print(f"found in bag: {w}")
    
    return np.array(bag)

  def predict_class(self, sentence):
    p = self.bow(sentence, self.words, show_details=False)
    res = self.model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
      return_list.append({'intent': self.classes[r[0]], 'probability': str(r[1])})

    return return_list
  
  def get_response(self, intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
      if i['tag'] == tag:
        result = np.random.choice(i['responses'])
        break

    return result

  def chatbot_response(self, msg):
      ints = self.predict_class(msg)
      res = self.get_response(ints, self.intents)
      return res

In [ ]:
bot = KickBot()
while True:
  msg = input('You: ')
  print('Bot:', bot.chatbot_response(msg))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Bot: I'm not sure I follow you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: I am here as KickVerse assistant!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: To sell on KickVerse, first create an account. Then, head to the seller dashboard, list your shoes, and wait for buyers to contact you!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: KickVerse is an online platform, so you can access it *here*!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: Like any other e-commerce platform, you can buy and sell products on KickVerse, but we specialize in shoes! You can also chat with sellers and leave reviews! What's more, we have an image recognition feature that allows you to search for shoes using images! Give it a try!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: I'm not sure I follow you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Bot: My pleasure!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: Like any other e-commerce platform, you can buy and sell products on KickVerse, but 